In [1]:
import os
import numpy as np 

import tensorflow as tf 
import tensorflow_hub as hub 
import tensorflow_datasets as tfds 

os.environ["TFHUB_MODEL_LOAD_FORMAT"] = "COMPRESSED"

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is ", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

Version:  2.3.1
Eager mode:  True
Hub version:  0.10.0
GPU is  available


In [2]:
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews",
    split=("train[:60%]", "train[60%:]", "test"),
    as_supervised=True
)

00:16, 1265.12 examples/s]
19508 examples [00:16, 1260.24 examples/s]
19648 examples [00:16, 1297.25 examples/s]
19791 examples [00:16, 1332.34 examples/s]
19936 examples [00:16, 1359.90 examples/s]
20073 examples [00:16, 1311.08 examples/s]
20211 examples [00:16, 1329.02 examples/s]
20345 examples [00:16, 1285.46 examples/s]
20475 examples [00:16, 1247.95 examples/s]
20612 examples [00:17, 1280.30 examples/s]
20744 examples [00:17, 1289.86 examples/s]
20883 examples [00:17, 1316.49 examples/s]
21017 examples [00:17, 1321.39 examples/s]
21150 examples [00:17, 1310.30 examples/s]
21282 examples [00:17, 1281.45 examples/s]
21415 examples [00:17, 1293.51 examples/s]
21545 examples [00:17, 1289.66 examples/s]
21678 examples [00:17, 1299.49 examples/s]
21809 examples [00:18, 1271.02 examples/s]
21941 examples [00:18, 1283.47 examples/s]
22071 examples [00:18, 1286.23 examples/s]
22208 examples [00:18, 1308.42 examples/s]
22340 examples [00:18, 1272.83 examples/s]
22468 examples [00:18, 1265

In [3]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [4]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0], dtype=int64)>

In [5]:
embedding = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(embedding, input_shape=[], dtype=tf.string, 
trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 50), dtype=float32, numpy=
array([[ 0.5423195 , -0.0119017 ,  0.06337538,  0.06862972, -0.16776837,
        -0.10581174,  0.16865303, -0.04998824, -0.31148055,  0.07910346,
         0.15442263,  0.01488662,  0.03930153,  0.19772711, -0.12215476,
        -0.04120981, -0.2704109 , -0.21922152,  0.26517662, -0.80739075,
         0.25833532, -0.3100421 ,  0.28683215,  0.1943387 , -0.29036492,
         0.03862849, -0.7844411 , -0.0479324 ,  0.4110299 , -0.36388892,
        -0.58034706,  0.30269456,  0.3630897 , -0.15227164, -0.44391504,
         0.19462997,  0.19528408,  0.05666234,  0.2890704 , -0.28468323,
        -0.00531206,  0.0571938 , -0.3201318 , -0.04418665, -0.08550783,
        -0.55847436, -0.23336391, -0.20782952, -0.03543064, -0.17533456],
       [ 0.56338924, -0.12339553, -0.10862679,  0.7753425 , -0.07667089,
        -0.15752277,  0.01872335, -0.08169781, -0.3521876 ,  0.4637341 ,
        -0.08492756,  0.07166859, -0.00670817,  0.12686075, -0.19326553,
 

In [6]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation="relu"))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 50)                48190600  
_________________________________________________________________
dense (Dense)                (None, 16)                816       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 48,191,433
Trainable params: 48,191,433
Non-trainable params: 0
_________________________________________________________________


In [7]:
model.compile(optimizer="adam",
loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
metrics=["accuracy"])

In [8]:
history = model.fit(train_data.shuffle(10000).batch(512),
epochs=10,
validation_data=validation_data.batch(512),
verbose=1)

Epoch 1/10
30/30 [==============================] - 3s 97ms/step - loss: 0.6887 - accuracy: 0.5335 - val_loss: 0.6191 - val_accuracy: 0.5560
Epoch 2/10
30/30 [==============================] - 3s 87ms/step - loss: 0.5535 - accuracy: 0.6575 - val_loss: 0.5114 - val_accuracy: 0.7290
Epoch 3/10
30/30 [==============================] - 3s 86ms/step - loss: 0.4245 - accuracy: 0.8035 - val_loss: 0.4142 - val_accuracy: 0.7906
Epoch 4/10
30/30 [==============================] - 3s 88ms/step - loss: 0.3078 - accuracy: 0.8793 - val_loss: 0.3511 - val_accuracy: 0.8487
Epoch 5/10
30/30 [==============================] - 3s 87ms/step - loss: 0.2240 - accuracy: 0.9199 - val_loss: 0.3211 - val_accuracy: 0.8613
Epoch 6/10
30/30 [==============================] - 3s 87ms/step - loss: 0.1638 - accuracy: 0.9472 - val_loss: 0.3080 - val_accuracy: 0.8698
Epoch 7/10
30/30 [==============================] - 3s 86ms/step - loss: 0.1190 - accuracy: 0.9658 - val_loss: 0.3076 - val_accuracy: 0.8728
Epoch 8/10
30

In [9]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
    print("%s: %.3f" % (name, value))

49/49 - 2s - loss: 0.3612 - accuracy: 0.8530
loss: 0.361
accuracy: 0.853
